# Testes e Otimização de Classificadores

In [1]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

dotenv_path = join(dirname('__file__'), '.env')

load_dotenv(dotenv_path)

ROOT_PATH = os.environ.get("ROOT_PATH")


In [3]:
import pandas as pd
import numpy as np

df = pd.read_parquet(f"{ROOT_PATH}/features/features.parquet")


## Definindo Classificadores

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    plt.xlim(-0.5, len(np.unique(y))-0.5)
    plt.ylim(len(np.unique(y))-0.5, -0.5)
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, auc, roc_curve, plot_confusion_matrix


def svm(params:dict, x:np.ndarray, y:np.ndarray):
    model = SVC(kernel='rbf',probability=True)
    y_pred = cross_val_predict(model, x, y, cv=5)
    m = confusion_matrix(y, y_pred)

    return m.astype('float')/m.sum(axis=1)[:, np.newaxis]








## Classificações Gerais

In [6]:
from sklearn.model_selection import train_test_split

x = df.loc[:,['var', 'skew', 'kur']]
y = df.loc[:, 'label']
    
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [7]:
params = {
    'kernel':'rbf', 
    'probability':True, 
    'C':100, 
    'gamma':0.00001
}

svm(params=params, x=x, y=y)

array([[0.40160643, 0.20682731, 0.        , 0.0753012 , 0.31626506],
       [0.27087794, 0.07815846, 0.        , 0.        , 0.6509636 ],
       [0.05825243, 0.        , 0.2815534 , 0.66019417, 0.        ],
       [0.10322581, 0.        , 0.40322581, 0.49354839, 0.        ],
       [0.07014028, 0.06713427, 0.        , 0.        , 0.86272545]])